In [ ]:
# Importación de las librerías necesarias para el entorno, manipulación de datos, redes neuronales y memoria
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

# Definir un conjunto de acciones discretas representativas para el entorno
DISCRETE_ACTIONS = [
    [-1.0, 0.5, 0.0],   # Giro fuerte izquierda con aceleración
    [-0.5, 0.5, 0.0],   # Giro suave izquierda con aceleración
    [0.0, 0.8, 0.0],    # Recto con mayor aceleración
    [0.5, 0.5, 0.0],    # Giro suave derecha con aceleración
    [1.0, 0.5, 0.0],    # Giro fuerte derecha con aceleración
    [0.0, 0.0, 0.3]     # Frenado más fuerte
]

# Función para verificar la disponibilidad de CUDA en PyTorch
def check_pytorch_cuda():
    """Check if PyTorch can access the GPU."""
    print("PyTorch CUDA availability:")
    print("CUDA available: ", torch.cuda.is_available())
    print("Device count: ", torch.cuda.device_count())
    if torch.cuda.is_available():
        print("CUDA device name: ", torch.cuda.get_device_name(0))

check_pytorch_cuda()

# Seleccionar el dispositivo CUDA si está disponible para el entrenamiento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definir la red Q con PyTorch para calcular los valores Q para cada acción
class QNetwork(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(QNetwork, self).__init__()
        # Definir capas convolucionales para extraer características de las observaciones del entorno
        self.conv1 = nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4)
        #32 salidas, num canales
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        # 32 entradas y 64 salidas 4*4 y paso 2 px
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        #64 y 64, 3*3 y paso 1 px

        # Calcular el tamaño de la salida de la última capa convolucional para pasar a las capas densas
        dummy_input = torch.zeros(1, *input_shape)
        #Al pasarlo a través de las capas convolucionales, obtenemos la cantidad exacta de características que resultan,
        conv_out_size = self._get_conv_output(dummy_input)
        
        # Definir las capas densas para calcular el valor Q final para cada acción
        self.fc1 = nn.Linear(conv_out_size, 512)
        self.fc2 = nn.Linear(512, num_actions)

    # Método para obtener el tamaño de salida después de las capas convolucionales
    def _get_conv_output(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        return x.view(1, -1).size(1)

    # Método para el paso hacia adelante (forward) en la red
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(x.size(0), -1)  # Aplanar la salida de la última capa convolucional
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Este proceso se basa en la política epsilon-greedy, que permite al agente equilibrar entre explorar nuevas acciones y explotar
def choose_action(state, q_network, epsilon):
    #state:parametro Actual
    # q_network: red neuronal para estimar los valores Q
    # epsilon: probabilidad de exploración para seleccionar una acción aleatoria
    #epsilon alto favorece la exploración, mientras que un valor bajo favorece la explotación
    # Selección de acción aleatoria para exploración
    if random.random() < epsilon:
        return random.choice(DISCRETE_ACTIONS)  # Exploración
    else:
        # Selección de la mejor acción estimada por la red para explotación
        with torch.no_grad():
            state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
            #Esto es necesario porque las redes neuronales de PyTorch suelen esperar un lote de datos de entrada.
            q_values = q_network(state_tensor)
            #q_network es un tensor que contiene los valores Q predichos para cada acción posible en DISCRETE_ACTIONS
            #ada valor Q representa la "calidad" o "utilidad
            action_idx = torch.argmax(q_values).item()
            #devuelve el índice de la acción con el valor Q más alto. Este índice representa la acción que maximiza la recompensa
            return DISCRETE_ACTIONS[action_idx]
            #lo cual devuelve el vector de acción que se utilizará en el entorno

# pues es la que se encarga de entrenar la red neuronal q_network utilizando los datos recolectados en la memoria de experiencias
def train_network(q_network, target_network, memory, gamma, batch_size, optimizer):
    # Asegurarse de que hay suficiente memoria para entrenar
    if len(memory) < batch_size:
        return
    # Seleccionar un batch aleatorio de la memoria
    batch = random.sample(memory, batch_size)
    states, actions, rewards, next_states, dones = zip(*batch)
    
    # Convertir las transiciones en tensores para el entrenamiento
    states = torch.tensor(states, dtype=torch.float32, device=device)
    actions = torch.tensor(actions, dtype=torch.long, device=device)
    rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
    next_states = torch.tensor(next_states, dtype=torch.float32, device=device)
    dones = torch.tensor(dones, dtype=torch.bool, device=device)

    # Calcular los valores Q actuales y los valores objetivo para la actualización
    q_values = q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
    #obteniendo un tensor de valores Q predichos para cada acción en cada estado
    #agrega una dimensión extra a actions para seleccionar las acciones correctas de los valores Q predichos
    #selecciona los valores Q de la red solo para las acciones que el agente realmente tomó en esos estados.
    # elimina la dimensión extra creada por gather, para que q_values sea un tensor unidimensional con los valores Q
    with torch.no_grad():
        next_q_values = target_network(next_states).max(1)[0]
        # Q futuros esperados el valor Q más alto posible para cada estado en next_states
    targets = rewards + gamma * next_q_values * (~dones)
    #calcular los valores objetivo se refiere a calcular la "meta" o 
    # "objetivo" que queremos que nuestra red neuronal aprenda para cada estado y acción

    # Calcular el error cuadrático medio y actualizar la red
    loss = nn.functional.mse_loss(q_values, targets)
    #Se utiliza la pérdida de error cuadrático medio
    #entre los valores Q actuales (q_values) y los valores objetivo (targets).
    #La pérdida mide qué tan lejos están las predicciones de q_network de los valores que queremos que aprenda
    optimizer.zero_grad()
    #borra los gradientes acumulados de la optimización anterior
    loss.backward()
    #calcula los gradientes de la pérdida con respecto a los parámetros de q_network usando retropropagación
    optimizer.step()
    #ajusta los pesos de q_network para minimizar la pérdida
    #Mejorar la precisión de q_network en predecir valores Q,

# Definir la función principal de entrenamiento del agente
#Su objetivo principal es permitir que el agente interactúe con el entorno, acumule experiencias, y mejore su red neuronal q_network
def train(episodes, max_steps=500, batch_size=64, gamma=0.99, epsilon_decay=0.995, epsilon_min=0.01, render=False):
    # Crear el entorno CarRacing en modo de renderizado
    env = gym.make("CarRacing-v3", render_mode='human')
    input_shape = (env.observation_space.shape[2],) + env.observation_space.shape[:2]
    #que será la forma de las imágenes del entorno que se usará como entrada de q_network
    num_actions = len(DISCRETE_ACTIONS)
    
    
    # Crear las redes Q y de objetivo y sincronizar pesos
    q_network = QNetwork(input_shape, num_actions).to(device)
    target_network = QNetwork(input_shape, num_actions).to(device)
    target_network.load_state_dict(q_network.state_dict())
    optimizer = optim.Adam(q_network.parameters(), lr=0.001)
    epsilon = 1.0
    memory = deque(maxlen=100000)
    reward_per_episode = np.zeros(episodes)
    
    # Bucle principal de entrenamiento por episodio
    for episode in range(episodes):
        if (episode+1) % 1 == 0:
            env.close()
            env = gym.make("CarRacing-v3", render_mode="human")
        else:
            env.close()
            env = gym.make("CarRacing-v3")

        state, _ = env.reset(seed=42)
        state = np.transpose(state, (2, 0, 1))
        episode_reward = 0
        done = False
        step_count = 0
        print(f"Comenzando episodio {episode + 1}")

        # Bucle de pasos dentro de cada episodio
        while not done and step_count < max_steps:
            action = choose_action(state, q_network, epsilon)
            action_idx = DISCRETE_ACTIONS.index(action)
            next_state, reward, done, _, _ = env.step(np.array(action, dtype=np.float32))
            #toma una accion y ejecuta un paso 
            
            # Añadir recompensa si el agente acelera y penalizar giros fuertes
            if action[1] > 0:
                reward += 1.0  # Recompensa adicional por acelerar
            if reward > 0 and action_idx == 2:
                reward += 0.5
            elif action_idx in [0, 4]:
                reward -= 5

            next_state = np.transpose(next_state, (2, 0, 1))
            # Reorganaizar las dimensiones de la observación del entorno
            memory.append((state, action_idx, reward, next_state, done))
            state = next_state
            episode_reward += reward
            step_count += 1
            train_network(q_network, target_network, memory, gamma, batch_size, optimizer)
            #train_network se llama para entrenar q_network con un batch de experiencias de 
            # memory, ajustando los pesos de q_network en función de los valores objetivo
        
        # Reducir epsilon gradualmente para favorecer la explotación
        epsilon = max(epsilon * epsilon_decay, epsilon_min)
        #Se reduce para que el agente explore menos con el tiempo
        reward_per_episode[episode] = episode_reward

        print( f"Recompensa del episodio : {episode_reward}")

        # Actualizar los pesos de la red de objetivo cada 10 episodios
        if (episode + 1) % 10 == 0:
            target_network.load_state_dict(q_network.state_dict())
            #e actualiza copiando los pesos de q_network. 
            # Esto estabiliza el entrenamiento al reducir la varianza en los valores Q

    # Graficar el desempeño del agente en términos de recompensa acumulada por episodio
    plt.plot(reward_per_episode)
    plt.xlabel("Episodios")
    plt.ylabel("Recompensa acumulada")
    plt.title("Desempeño del agente durante el entrenamiento")
    plt.show()

# Llamar a la función de entrenamiento con 1000 episodios
train(episodes=100)

PyTorch CUDA availability:
CUDA available:  False
Device count:  0
Comenzando episodio 1
Recompensa del episodio : -269.49293286218943
Comenzando episodio 2
Recompensa del episodio : -339.895759717315
Comenzando episodio 3
Recompensa del episodio : -236.19151943462825
Comenzando episodio 4
Recompensa del episodio : -264.7964664310949
Comenzando episodio 5
